In [1]:
import pandas as pd
dtypes = {"date": "str"}
parse_dates = ["date"]
df = pd.read_csv("covidsg.csv", dtype=dtypes, parse_dates=parse_dates)
df.head()

,case,date,type,race,age,gender,hospital
0,#1114,2020-04-03,Local,Singapore Citizen,28,M,CGH
1,#1113,2020-04-03,Local,Bangladeshi (Singapore Work Pass holder),42,M,NTFGH
2,#1112,2020-04-03,Local,Burmese (Long Term Pass holder),32,M,KTPH
3,#1111,2020-04-03,Local,Burmese (Long Term Pass holder),29,M,KTPH
4,#1110,2020-04-03,Local,Bangladeshi (Long Term Pass holder),27,M,KTPH


In [2]:
data = [line.rstrip("\n") for line in open("./covidsg_local.csv")]
clusters = {}
for line in data:
    line = line.split(",")
    name = str(line[0])
    cases = []
    for i in line[1:]:
        if len(i) > 0:
            cases.append(int(i))
    clusters[name] = cases

In [3]:
from pyvis.network import Network

In [4]:
net = Network(height="800px", width="100%")

for name in list(clusters.keys()):
    net.add_node(name)
    for case in clusters[name]:
        net.add_node(case)
        net.add_edge(case, name)
        
def get_desc(node_id):
    case_data = df.loc[df["case"] == node_id]
    origin = case_data["type"].values[0]
    age = str(case_data["age"].values[0])
    race = case_data["race"].values[0]
    gender = case_data["gender"].values[0]
    hospital = case_data["hospital"].values[0]
    desc = ",".join([origin,age,race,gender,hospital])
    return desc
        
for node in net.nodes:
    try:
        node_id = "#"+str(int(node["id"]))
        try:
            case_desc = get_desc(node_id)
            node["title"] = case_desc
            case_data = case_desc.split(",")
            if case_data[0] == "Imported":
                node["color"] = "#00C11C"
            else:
                node["color"] = "#C10000"
        except:
            node["title"] = "No data"
            node["color"] = "#C10000"
        node["size"] = 5
    except Exception as e:
        node["color"] = "#660033"
        node["size"] = 10

In [5]:
net.toggle_physics(True)
net.show("clusters.html")